In [2]:

import os
import time
import scipy.io as scio
import datetime
import re
import matplotlib.pyplot as plt
import numpy as np
import pylab
import os
import csv
from skimage import transform, filters
from textwrap import wrap
import cv2


import torch
from torch.autograd import Variable
from torchvision import models
import sys
import matplotlib as mpl
#mpl.use('Agg')


import torchvision
import torchvision.transforms as transforms

from typing import Tuple
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

In [7]:
# bring your colab code here, 

from IGos_utils import *

igos' utils.py file is successfully imported


In [4]:
# i want you to clear up this code, start with the obvious parts

def Get_blurred_img(img, model, Gaussian_param = [51, 50], use_cuda = 1):
    ########################
    # Generate blurred images as the baseline

    # Parameters:
    # -------------
    # input_img: the original input image
    # img_label: the classification target that you want to visualize (img_label=-1 means the top 1 classification label)
    # model: the model that you want to visualize
    # resize_shape: the input size for the given model
    #
    # 

    #output
    # ----
    # original_img: resized numpy image
    # img:  - BGR -,   np.float32()/255
    # blurred_img -BGR - np.float32()/255
    # logitori: class score
    ####################################################

    
    print("before the thing", img.dtype, img.shape, img.min(), img.max())
    
    # img = np.float32(img) / 255
   
    
    Kernelsize = Gaussian_param[0]
    SigmaX = Gaussian_param[1]
    blurred_img = cv2.GaussianBlur(img, (Kernelsize, Kernelsize), SigmaX)

    img_torch = preprocess_image(img, use_cuda, require_grad = False)

    ori_output = model(img_torch)

    # compute the outputs for the original image and the blurred image
    if use_cuda:
        logitori = ori_output.data.cpu().numpy().copy().squeeze()
    else:
        logitori = ori_output.data.numpy().copy().squeeze()


    return  img, blurred_img, logitori


In [5]:
# input_path = './Images/'
# if not os.path.exists(input_path):
#     os.makedirs(input_path)


output_path = './Results/VIDEO/'
if not os.path.isdir(output_path):
    os.makedirs(output_path)


model = load_model_new(use_cuda=use_cuda, model_name='vgg19')  #

#############################################
# imgname  = "ILSVRC2012_val_00001003.JPEG"
# input_img = input_path + imgname

# Define a transform to convert images to tensors and normalize them
transform = transforms.Compose([
                transforms.ToTensor()
                                ])

# Load the CIFAR-10 training set
trainset = torchvision.datasets.CIFAR10(root='~/data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=False)

# Define class labels
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Get one batch (the first image)
dataiter = iter(trainloader)
images, labels = next(dataiter)



input_img =  images.squeeze().permute(1,2,0).cpu().numpy()
##############################################





img_label = -1

img, blurred_img, logitori = Get_blurred_img(input_img, model,
                                                Gaussian_param=[51, 50], use_cuda=use_cuda)


# mask, upsampled_mask, imgratio, curvetop, curve1, curve2, category = Integrated_Mask(img, blurred_img, model,
#                                                                                             img_label,
#                                                                                             max_iterations=15,
#                                                                                             integ_iter=20,
#                                                                                             tv_beta=2,
#                                                                                             l1_coeff=0.01 * 100,
#                                                                                             tv_coeff=0.2 * 100,
#                                                                                             size_init=28,
#                                                                                             use_cuda=0)



/home/bm1/miniconda3/envs/I_GOS/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/home/bm1/miniconda3/envs/I_GOS/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Files already downloaded and verified
before the thing float32 (32, 32, 3) 0.0 1.0
